# ***CSB352: Data Mining***

Instructor : [Dr. Chandra Prakash]

# **LAB Assignment 7: Association Mining**

## Assigning Date: 15-Feb-2021

## Due Date: 21-Feb-2021


# **Student Name: Rohit Byas**

# **Roll No: 181210043**

In [7]:
try:
    from google.colab import drive
    %tensorflow_version 2.x
    COLAB = True
    print("Assignment 7 - Association Mining")
    print("Note: using Google CoLab")
except:
    print("Assignment 7 - Association Mining")
    print("Note: not using Google CoLab")
    COLAB = False

Assignment 7 - Association Mining
Note: using Google CoLab


In [8]:
print("Name: Rohit Byas")
print("Roll Number : 181210043")

Name: Rohit Byas
Roll Number : 181210043


In [9]:
from datetime import date

today = date.today()
print("Current Date:", today)

Current Date: 2021-02-22


In [10]:
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%H:%M:%S")
print("Current Time:", dt_string)

Current Time: 04:16:27


## ***4.0.1 Task 0: Getting to Know Your Data***

## **Read Dataset [ L7_Groceries.csv ] from the link from LAB 1**

In [11]:
import itertools
import numpy as np
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [3]:
#Downloading the dataset:
!gdown --id "1AbvTYu_UDv_IKqmh0CNLfib47aqqBt2Z"

Downloading...
From: https://drive.google.com/uc?id=1AbvTYu_UDv_IKqmh0CNLfib47aqqBt2Z
To: /content/L7_Groceries.csv
100% 501k/501k [00:00<00:00, 66.4MB/s]


In [12]:
#Storing all the transactions from the dataset in a list 
list_of_transactions = []

for line in open("L7_Groceries.csv"):
    transaction = line.split(",")
    transaction[len(transaction)-1] = transaction[len(transaction)-1].replace("\n", "")
    list_of_transactions.append(transaction)

In [13]:
#Printing the list of transactions from the dataset:
pp.pprint(list_of_transactions)

Streaming output truncated to the last 5000 lines.
        'whipped/sour cream',
        'processed cheese',
        'frozen fruits'],
    ['soda', 'canned beer', 'newspapers'],
    ['soda', 'misc. beverages', 'waffles', 'shopping bags'],
    ['grapes', 'berries', 'butter milk', 'yogurt', 'pot plants'],
    ['specialty chocolate'],
    [   'pip fruit',
        'ice cream',
        'bottled water',
        'soda',
        'fruit/vegetable juice',
        'bottled beer'],
    ['chewing gum', 'chocolate', 'specialty chocolate'],
    [   'frankfurter',
        'chicken',
        'citrus fruit',
        'root vegetables',
        'packaged fruit/vegetables',
        'domestic eggs',
        'brown bread',
        'artif. sweetener',
        'bottled water',
        'waffles',
        'napkins'],
    [   'frankfurter',
        'root vegetables',
        'onions',
        'whole milk',
        'curd',
        'butter milk',
        'frozen meals',
        'oil',
        'soda',
        'fruit

In [14]:
#Function for storing the frequency count of each item in item_counts dictionary

def return_item_freq(list_of_transactions):
    item_count = {}
    
    for transaction in list_of_transactions:
        for item in transaction:
            if frozenset([item]) in item_count:
                item_count[frozenset([item])] = item_count[frozenset([item])]+1
            else:
                item_count[frozenset([item])] = 1
                
    return item_count

In [15]:
#Storing the frequency count in item_count
item_count = return_item_freq(list_of_transactions)

In [16]:
print("Number of unique items:", len(item_count))

Number of unique items: 169


In [17]:
#Displaying the itemset with its corresponding frequency (support count):
item_count

{frozenset({'citrus fruit'}): 814,
 frozenset({'semi-finished bread'}): 174,
 frozenset({'margarine'}): 576,
 frozenset({'ready soups'}): 18,
 frozenset({'tropical fruit'}): 1032,
 frozenset({'yogurt'}): 1372,
 frozenset({'coffee'}): 571,
 frozenset({'whole milk'}): 2513,
 frozenset({'pip fruit'}): 744,
 frozenset({'cream cheese '}): 390,
 frozenset({'meat spreads'}): 42,
 frozenset({'other vegetables'}): 1903,
 frozenset({'condensed milk'}): 101,
 frozenset({'long life bakery product'}): 368,
 frozenset({'butter'}): 545,
 frozenset({'rice'}): 75,
 frozenset({'abrasive cleaner'}): 35,
 frozenset({'rolls/buns'}): 1809,
 frozenset({'UHT-milk'}): 329,
 frozenset({'bottled beer'}): 792,
 frozenset({'liquor (appetizer)'}): 78,
 frozenset({'pot plants'}): 170,
 frozenset({'cereals'}): 56,
 frozenset({'white bread'}): 414,
 frozenset({'bottled water'}): 1087,
 frozenset({'chocolate'}): 488,
 frozenset({'curd'}): 524,
 frozenset({'flour'}): 171,
 frozenset({'dishes'}): 173,
 frozenset({'beef'}

## **4.1 TASK 1. Apriori Algorithm**

In [18]:
def calculate_support_count(item_set, list_of_transactions):
    count = 0
    for transaction in list_of_transactions:
        if set(item_set).issubset(set(transaction)):
            count += 1
    return count

In [19]:
def get_association_rules(freq_item_sets, dataset, min_confidence):
    association_rules = []
    for item_set in freq_item_sets.keys():
        s = list(item_set)
        subsets = list(itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(1, len(s)+1)))
        for sub_item_set in subsets:
            support = calculate_support_count(item_set, dataset)/len(dataset)
            rhs = set(item_set).difference(sub_item_set)
            if(len(rhs)==0):
                break
            lhs = sub_item_set
            support_of_lhs = calculate_support_count(frozenset(lhs), dataset)/len(dataset)

            confidence = (freq_item_sets[item_set]/len(dataset))/support_of_lhs
            if(confidence>min_confidence):
                support_of_rhs = calculate_support_count(frozenset(rhs), dataset)/len(dataset)
                lift = confidence/support_of_rhs
                rule = {}
                rule['LHS'] = set(lhs)
                rule['RHS'] = rhs
                rule['Confidence'] = confidence
                rule['Lift'] = lift
                rule['Support'] = support
                association_rules.append(rule)
    return association_rules

In [20]:
#Defining my own apriori algorithm
def my_apriori(dataset, min_support, min_confidence):
    min_support = min_support*len(dataset)
    #Extracting all the items with support count more than min_support
    #And also sorting the dictionary in descending order of their frequency
    item_count = dict(sorted([elem for elem in return_item_freq(dataset).items() if elem[1]>=min_support],
                             key=lambda item: item[1],
                             reverse=True))
    keys = [frozenset(elem[0]) for elem in item_count.items()]
    next_keys = []
    
    coupling_count = 2
    while True:
        next_keys = []
        for subset in itertools.combinations(keys, coupling_count):
            next_keys.append(frozenset(np.array([list(elem) for elem in subset]).flatten()))
        coupling_count+=1
        next_item_count = {}
        
        for key in next_keys:
            next_item_count[key] = calculate_support_count(key, dataset)
        
        next_item_count = dict(sorted([elem for elem in next_item_count.items() if elem[1]>=min_support],
                             key=lambda item: item[1],
                             reverse=True))
        
        if len(next_item_count)==0:
            break
        item_count = next_item_count
        keys = [elem[0] for elem in item_count.items()]
    freq_item_sets = item_count
    print("Frequent Item Sets:")
    pp.pprint(freq_item_sets)
    
    #Association rules:
    association_rules = get_association_rules(freq_item_sets, dataset, min_confidence)
                
    return association_rules

In [21]:
list_of_rules = my_apriori(list_of_transactions, min_support=0.03, min_confidence=0.3)

Frequent Item Sets:
{   frozenset({'whole milk', 'other vegetables'}): 736,
    frozenset({'whole milk', 'rolls/buns'}): 557,
    frozenset({'yogurt', 'whole milk'}): 551,
    frozenset({'whole milk', 'root vegetables'}): 481,
    frozenset({'root vegetables', 'other vegetables'}): 466,
    frozenset({'yogurt', 'other vegetables'}): 427,
    frozenset({'rolls/buns', 'other vegetables'}): 419,
    frozenset({'tropical fruit', 'whole milk'}): 416,
    frozenset({'whole milk', 'soda'}): 394,
    frozenset({'rolls/buns', 'soda'}): 377,
    frozenset({'tropical fruit', 'other vegetables'}): 353,
    frozenset({'bottled water', 'whole milk'}): 338,
    frozenset({'yogurt', 'rolls/buns'}): 338,
    frozenset({'whole milk', 'pastry'}): 327,
    frozenset({'other vegetables', 'soda'}): 322,
    frozenset({'whipped/sour cream', 'whole milk'}): 317,
    frozenset({'rolls/buns', 'sausage'}): 301,
    frozenset({'citrus fruit', 'whole milk'}): 300,
    frozenset({'whole milk', 'pip fruit'}): 296}


In [22]:
df = pd.DataFrame(list_of_rules)
df = df.sort_values(by=['Lift'], ascending=False)
df = df.reset_index().drop(columns=['index'])
df

,LHS,RHS,Confidence,Lift,Support
0,{root vegetables},{other vegetables},0.434701,2.246605,0.047382
1,{sausage},{rolls/buns},0.325758,1.771048,0.030605
2,{tropical fruit},{other vegetables},0.342054,1.767790,0.035892
3,{whipped/sour cream},{whole milk},0.449645,1.759754,0.032232
4,{root vegetables},{whole milk},0.448694,1.756031,0.048907
5,{yogurt},{other vegetables},0.311224,1.608457,0.043416
6,{tropical fruit},{whole milk},0.403101,1.577595,0.042298
7,{yogurt},{whole milk},0.401603,1.571735,0.056024
8,{pip fruit},{whole milk},0.397849,1.557043,0.030097
9,{other vegetables},{whole milk},0.386758,1.513634,0.074835


## **4.2 TASK 2. Frequent Pattern Growth Algorithm**

In [23]:
#Creating a trie data structure:
class Tree:
    def __init__(self, item, parent):
        self.item = item
        self.parent = parent
        if item is not None:
            self.frequency = 1
        else:
            self.frequency = 0
        self.children = []
        
    def add_child(self, itemset):
        #itemset must be in decreasing order of their frequency
        if len(itemset)>0:
            current_item = itemset[0]
            if current_item == self.item:
                self.frequency+=1
                if len(itemset)>1:
                    self.add_child(itemset[1:])
            else:
                #Search for child == current_item
                found = False
                for child in self.children:
                    if child.item == current_item:
                        child.add_child(itemset)
                        found = True
                        break
                #If no such child, create a new child
                if not found:
                    new_child = Tree(current_item, self)
                    self.children.append(new_child)
                    if len(itemset)>1:
                            new_child.add_child(itemset[1:])
                            
    def print_all_children(self):
        print("{item:", self.item, ", freq:",self.frequency, "}")
        if len(self.children)>0:
            for child in self.children:
                child.print_all_children()
       
    #wrong implementation
    def get_all_paths(self, item):
        set_of_paths = []
        if(self.item==item):
            freq = self.frequency
            #recursively get its path by traversing through its parents:
            path = []
            node = self.parent
            while node.item is not None:
                path.append(node.item)
                node = node.parent
            if(len(path)>0):
                path.sort(reverse=True)
                set_of_paths.append({'path': path, 'frequency': freq})
        else:
            if len(self.children)>0:
                for child in self.children:
                    set_of_paths += child.get_all_paths(item)
        return set_of_paths

In [24]:
def get_sorted_dataset(dataset, min_support):
    item_count = dict(sorted([elem for elem in return_item_freq(dataset).items() if elem[1]>min_support],
                             key=lambda item: item[1],
                             reverse=True))
    new_sorted_dataset = []
    for transaction in dataset:
        new_transaction = sorted([item for item in transaction if frozenset({item}) in item_count.keys()], key = lambda item: item_count[frozenset({item})], reverse=True)
        if len(new_transaction)>0:
            new_sorted_dataset.append(new_transaction)
    return new_sorted_dataset

In [25]:
def my_FPG(dataset, min_support, min_confidence):
    min_support = min_support*len(dataset)
    new_sorted_dataset = get_sorted_dataset(dataset, min_support)
    count = 0
    root = Tree(None, None)
    for transaction in new_sorted_dataset:
        root.add_child(transaction)
    #root.print_all_children()
    item_count = dict(sorted([elem for elem in return_item_freq(dataset).items() if elem[1]>min_support],
                             key=lambda item: item[1],
                             reverse=True))
    #print(item_count)
    list_of_all_paths = []
    items = [list(elem)[0] for elem in list(item_count.keys())]
    #pp.pprint(item_count)
    for item in items:
        paths = root.get_all_paths(item)
        if(len(paths)>0):
            list_of_all_paths.append({
                "item": item,
                "paths": paths
            })
    pp.pprint(list_of_all_paths)

In [26]:
my_FPG(list_of_transactions, 0.03, 0.3)

Streaming output truncated to the last 5000 lines.
                                     'long life bakery product',
                                     'citrus fruit']},
                     {'frequency': 1, 'path': ['pastry', 'citrus fruit']},
                     {   'frequency': 1,
                         'path': ['pastry', 'hamburger meat', 'beef']},
                     {   'frequency': 1,
                         'path': [   'white bread',
                                     'whipped/sour cream',
                                     'tropical fruit',
                                     'long life bakery product',
                                     'curd',
                                     'citrus fruit',
                                     'bottled water']},
                     {   'frequency': 1,
                         'path': ['shopping bags', 'bottled water']},
                     {   'frequency': 1,
                         'path': ['dessert', 'bottled water', '

## **4.3 TASK 3: Compare the results of your funcitons for both algorithm with the inbuild/pre-build packages respectively.**

In [27]:
from mlxtend.frequent_patterns import apriori,association_rules
from mlxtend.preprocessing import TransactionEncoder

In [28]:
te = TransactionEncoder()
te_ary = te.fit(list_of_transactions).transform(list_of_transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [29]:
frequent_itemsets = apriori(df, min_support=0.03, use_colnames=True)

rules = association_rules(frequent_itemsets, metric ="confidence", min_threshold = 0.3) 
rules = rules.sort_values('lift', ascending =False)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(root vegetables),(other vegetables),0.108998,0.193493,0.047382,0.434701,2.246605,0.026291,1.426693
8,(sausage),(rolls/buns),0.093950,0.183935,0.030605,0.325758,1.771048,0.013324,1.210344
3,(tropical fruit),(other vegetables),0.104931,0.193493,0.035892,0.342054,1.767790,0.015589,1.225796
12,(whipped/sour cream),(whole milk),0.071683,0.255516,0.032232,0.449645,1.759754,0.013916,1.352735
10,(root vegetables),(whole milk),0.108998,0.255516,0.048907,0.448694,1.756031,0.021056,1.350401
5,(yogurt),(other vegetables),0.139502,0.193493,0.043416,0.311224,1.608457,0.016424,1.170929
11,(tropical fruit),(whole milk),0.104931,0.255516,0.042298,0.403101,1.577595,0.015486,1.247252
13,(yogurt),(whole milk),0.139502,0.255516,0.056024,0.401603,1.571735,0.020379,1.244132
7,(pip fruit),(whole milk),0.075648,0.255516,0.030097,0.397849,1.557043,0.010767,1.236375
4,(other vegetables),(whole milk),0.193493,0.255516,0.074835,0.386758,1.513634,0.025394,1.214013


### Your Learning and observation

**Observation**

we came to know that people buy milk products very frequently and together i.e. 
1. milk, yogurt, cheese together
2. onion with vegetables.
3. soda with margarine
etc.

**Learning**

In this assignment, I have learnt to create functions to implement these algorithms practically.


After creating my functions, I also compared the results with the in-build python packages to understand better